In [1]:
import numpy as np
import random
import pandas as pd
import math


In [2]:
for i in range(125):
    df = pd.read_csv(f'./finaldatasetgz/googleclusterdata{i}.csv.gz')
    
    if i==0:
        data= df.copy()

    else :
        data= pd.concat([data,df],axis=0,ignore_index=True)

    print(i)    

print(df)     

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
        Unnamed: 0     timestamp sizecpu sizeram  timeslot  gs   ds  cloud
0         47987876  2.479339e+12       s       l     688.0   1  150      5
1         47987877  2.479339e+12       s       l     688.0   1  300      3
2         47987878  2.479339e+12       s       l     688.0   1  150      0
3         47987879  2.479339e+12       s       l     688.0   1  300      3
4         47987880  2.479339e+12       s       l     688.0   2  150      8
...            ...           ...     ...     ...       ...  ..  ...    ...
386994    48374870  2.506197e+12       m       s     696.0   2  150      7
386995    

In [3]:
data

,Unnamed: 0,timestamp,sizecpu,sizeram,timeslot,gs,ds,cloud
0,0,0.000000e+00,l,l,0.0,1,150,4
1,1,0.000000e+00,l,l,0.0,2,300,8
2,2,0.000000e+00,m,l,0.0,1,150,9
3,3,0.000000e+00,m,l,0.0,2,300,0
4,4,0.000000e+00,m,l,0.0,2,300,5
...,...,...,...,...,...,...,...,...
48374870,48374870,2.506197e+12,m,s,696.0,2,150,7
48374871,48374871,2.506197e+12,m,s,696.0,2,150,0
48374872,48374872,2.506198e+12,m,s,696.0,1,150,3
48374873,48374873,2.506200e+12,m,s,696.0,1,150,6


In [4]:
class jobreq:
    #S types of Jobs serviced at each cloud, 
    # each specified by a three-tuple < ms,gs,ds>    
    def __init__(self,cloud,ms,gs,ds,jobtype):
        #request for cloud
        self.cloud = cloud
        #type of required VM
        self.ms=ms
        # # of VMs required+
        self.gs=gs
        # SLA in terms of Response time
        self.ds=ds
        #job type S
        self.jobtype = jobtype    

In [5]:
# def randomsamplingHTD(length,clouds,requireddata):
#     a=2.0
#     dist = np.random.power(a, size = length)
#     scaled_dist = np.round(dist*9).astype(int)
#     values = np.random.choice(scaled_dist,size=length)
#     sample = pd.DataFrame(values,columns=['cloud'],index = list(requireddata.index.values))
#     sample = pd.concat([requireddata,sample],axis=1)
    
#     return sample
    

In [6]:
def jobgenerator(df,timeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM):
    requests = []
#     requireddata =df.loc[df['timeslot'] == timeslot+1]
    condition = df["timeslot"] == timeslot+1
    requireddata = df[condition].copy()
#     requireddata['cloud'] = np.random.randint(10,size = requireddata.shape[0])
    
    for i in range(len(requireddata)):
        m1=requireddata['sizeram'].iloc[i]
        m2=requireddata['sizecpu'].iloc[i]
        m = f'{m1}{m2}'
        g = requireddata['gs'].iloc[i]
        d = requireddata['ds'].iloc[i]
        c = requireddata['cloud'].iloc[i]
        t = (4*possibleVMtypes.index(m))+(2*possiblenoofVM.index(g))+possibleDS.index(d)
        requests.append(jobreq(c,m,g,d,t))
    return requests

In [7]:
class CSPs:
  
    def __init__(self,name,nMI,betaIT,cuspay,typeofServiceS=24):
        self.nMI=nMI
        self.name=name
        #buy-bid to be submitted to auctioneer    
        self.buybid={}
        #sell-bid to be submitted to auctioneer
        self.sellbid={}
         #queue maintained by cloud
        self.typeofServiceS=typeofServiceS
        self.quelen = [0 for i in range(typeofServiceS)]
        self.possibleVMtype = ['ss','sm','sl','ls','lm','ll']
        self.possibleDS = [150,300]
        self.possiblenoofVM = [1,2]
        self.V=3000000
        #virtual queue maintained by cloud
        self.vQue = [0 for i in range(typeofServiceS)]
        # # of jobs dropped
        self.D = [0 for i in range(typeofServiceS)]
        #cost of running the server
        self.betaIT=betaIT
#         self.cusPay = [0.02,0.02,0.02,0.04,0.04,0.04]
        self.cusPay = cuspay
        self.cMI = [30,30,30,10,10,10]
        self.allService = self.obtainServiceSdesc()
        self.maxNoofJobdrop=1000
        self.xi=cuspay
        self.pays = [0 for i in range(5)]
        self.totalpay = 0
        
        
    def obtainServiceSdesc(self):
        possibleServiceS ={}
        x=0
        for i in possibleVMtypes:
            for j in possiblenoofVM:
                for k in possibleDS:
                    possibleServiceS[x]=[i,j,k]
                    x=x+1     
        return possibleServiceS


    def initqueue(self,rsit):
        for i in rsit[self.name]:
            self.quelen[i.jobtype] = self.quelen[i.jobtype]+1
            self.updatepayoff(0,(i.gs*self.cusPay[self.possibleVMtype.index(i.ms)]))
                    
    def keyAddition(self,dicts,key,value):
        if key not in list(dicts.keys()):
            temp=[]
            temp.append(value)
            dicts[key]=temp
        else:
            dicts[key].append(value)
        return dicts
    
    
    def getBids(self):
        self.wsdashi={}
        for i in range(len(self.quelen)):        
            if self.quelen[i]==0:
                self.wsdashi = self.keyAddition(self.wsdashi,self.allService[i][0],0)
                continue        
            else:
                self.wsdashi = self.keyAddition(self.wsdashi,self.allService[i][0],(self.quelen[i]+self.vQue[i])/self.allService[i][1])
        
        self.sStarM={}      
        for key,value in self.wsdashi.items():
            index = (self.typeofServiceS/len(self.possibleVMtype))*self.possibleVMtype.index(key)+value.index(max(value))
            self.sStarM[key]=index
            self.buybid[key]=max(value)/self.V
            self.sellbid[key]=max(max(value)/self.V,self.betaIT/self.cMI[self.possibleVMtype.index(key)])
#         print("Buybid:",self.buybid) 
#         print("Sellbid:",self.sellbid)
#         print("sStar:",self.sStarM)
#         print("wsdashi:",self.wsdashi)

    
    def notosell(self,ind):
        return self.cMI[ind]*self.nMI[ind]
      
    def queueupdate(self,muSIJ,rsit):
        for services in range(len(muSIJ)):
            summation = sum(muSIJ[services][self.name])
            if summation >= self.quelen[services]:
                self.quelen[services]=0
            else:
                self.quelen[services] = self.quelen[services] - summation 
        self.initqueue(rsit)
        
    def vqueueupdate(self,muSIJ,csps):
        for services in range(len(muSIJ)):
            msInd=self.possibleVMtype.index(self.allService[services][0])
            ds = self.allService[services][2]
            gs=self.allService[services][1]
            
            if ds==150:
                epsilonS = 1000
            elif ds==300:
                epsilonS=500
            
            if self.quelen[services]==0:
                t=1
            else:
                t=0
            temp = self.vQue[services] + (1-t)*(epsilonS-sum(muSIJ[services][self.name])) - t*sum([(i.cMI[msInd]*i.nMI[msInd])/gs for i in csps])
            self.vQue[services]=max(0,temp)
            
            
    def dropjobs(self):
        for services in range(self.typeofServiceS):
            tempxi =self.xi[self.possibleVMtype.index(self.allService[services][0])]*self.allService[services][1]
            if (self.quelen[services]+self.vQue[services]) > (self.V*tempxi):
                self.D[services]=self.D[services]+self.maxNoofJobdrop
                self.quelen[services]=max(0,self.quelen[services]-self.maxNoofJobdrop)
                self.vQue[services]=max(0,self.vQue[services]-self.maxNoofJobdrop)
                self.updatepayoff(3,tempxi*1000)
                            
    def updatepayoff(self,ind,pay):
        self.pays[ind]= self.pays[ind]+pay
        
    def provisioningcost(self,nmi):
        for keys,values in nmi.items():
            self.pays[2]= self.pays[2]+ self.betaIT*values[self.name]
            
    def totalpayment(self):
        self.totalpay = self.pays[0]+self.pays[1]-self.pays[2]-self.pays[3]-self.pays[4]
            
    def dropinend(self):
        for services in range(self.typeofServiceS):
#             print(self.quelen[services]*self.cusPay[self.possibleVMtype.index(self.allService[services][0])])
            self.pays[0]= self.pays[0]- (self.quelen[services]*self.cusPay[self.possibleVMtype.index(self.allService[services][0])]*self.allService[services][1])
    

        
        
        

In [8]:
class auctioneer:
         
    Buybid={}
    Sellbid={}
    possibleVMtype = ['ss','sm','sl','ls','lm','ll']
    
    def __init__(self,CSP):
        self.csps = CSP
        
    def allbidsandasks(self):
        for i in self.possibleVMtype:
            self.Buybid[i]={}
            self.Sellbid[i]={}
            for c in self.csps:
                self.Buybid[i][c.name]=c.buybid[i]
                self.Sellbid[i][c.name]=c.sellbid[i]
    
    def printbids(self):
        print("Buybids:",self.Buybid)
        print("SellBids:",self.Sellbid)
        
    def sortBid(self):
        for key in self.Buybid:
            self.Buybid[key]=dict(sorted(self.Buybid[key].items(),reverse=True, key=lambda item: item[1]))
            
        for key in self.Sellbid:
            self.Sellbid[key]=dict(sorted(self.Sellbid[key].items(),key=lambda item: item[1]))
            
            
    def winnerDetermination(self):
        self.winningBuyer={}
        self.winningSeller={}
        for key,value in self.Buybid.items():
            self.winningBuyer[key]={}
            s=sorted(set(list(value.values())))
            self.winningBuyer[key][list(value.items())[0][0]]=s[-2]
        print("WinningBuyer:",self.winningBuyer)
        
        for key,value in self.Sellbid.items():
            setofwinningseller=[]
            clearingprice = list(self.winningBuyer[key].values())[0]
            clearingsellprice=0

            for keys,values in value.items():
                if values <= clearingprice:
                    setofwinningseller.append(keys)
                    clearingsellprice=values
                    
            if len(setofwinningseller) != 0:   
                setofwinningseller.pop()
            self.winningSeller[key]=[]
            self.winningSeller[key].append(setofwinningseller)
            self.winningSeller[key].append(clearingsellprice)            
        print("WinningSeller:",self.winningSeller)
            
        
        # of VMs sold from cloud j to cloud i
        
    def determineAlpha(self):
        self.alphaMIJ = {}
        for size in self.possibleVMtype:
            self.alphaMIJ[size]=[[0 for i in range(len(self.csps))] for j in range(len(self.csps))]
            temp =[0 for x in range(len(self.csps))]
            for i in range(len(self.csps)):
                if i in self.winningSeller[size][0]:
                    temp[i]=self.csps[i].notosell(self.possibleVMtype.index(size))
            self.alphaMIJ[size][list(self.winningBuyer[size].keys())[0]]= temp.copy()
#         print("# of VMs sold j->i alphaMIJ:",self.alphaMIJ)
    
    def determineprice(self):
        for keys,values in self.winningSeller.items():
            winnername = list(self.winningBuyer[keys].keys())[0]
            temp = 0
            if len(values[0])>0:
                for i in values[0]:
                        self.csps[i].pays[1]=self.csps[i].pays[1] + self.alphaMIJ[keys][winnername][i]*values[1]
                        temp = temp + self.alphaMIJ[keys][winnername][i]
                        
            self.csps[winnername].pays[4] = self.csps[winnername].pays[4]+ temp * self.winningBuyer[keys][winnername]
                    
            
    

In [9]:
def determineMu(csps,auc):
    possibleService=csps[0].obtainServiceSdesc()
    
    muSIT = [[[0 for i in csps] for j in csps] for k in range(csps[0].typeofServiceS)]

    for key,value in auc.alphaMIJ.items():
        winningBuy=list(auc.winningBuyer[key].keys())
        winningSell=list(auc.winningSeller[key][0])
        servtype = int(csps[winningBuy[0]].sStarM[key])
        gs = possibleService[servtype][1]
        msInd= csps[0].possibleVMtype.index(key)
        value = [[x//gs for x in y] for y in value] 
        for i in csps:
            if i.name in winningBuy or i.name not in winningSell:
                if ((i.quelen[servtype]+i.vQue[servtype])/gs) > (i.V*i.betaIT/i.cMI[msInd]) :
                    value[i.name][i.name]=i.notosell(msInd)//gs
        muSIT[servtype]=value

    return muSIT

In [10]:
def noOfactivatedServers(muSIT,csps,auc):
    possibleService=csps[0].obtainServiceSdesc()
    activeNMI={}
    sumAlpha={}

    for key,value in auc.alphaMIJ.items():
        temp=[0 for i in csps]
        sumAlpha[key]=[]
        for cloud in value:
            temp = [(temp[i]+cloud[i]) for i in range(len(csps))]
        sumAlpha[key]=temp.copy()    
        
    sumMuSII  = {} 
    mS = -1
    for servtype in range(len(muSIT)):
        if mS != possibleService[servtype][0]:
            mS=possibleService[servtype][0]
            sumMuSII[mS]=[]
            temp=[0 for i in csps]

        for j in range(len(muSIT[servtype])):
            temp[j] = temp[j]+muSIT[servtype][j][j]*possibleService[servtype][1]
            
        sumMuSII[mS]= temp.copy()
    
    for key,value in sumMuSII.items():
        activeNMI[key]=[(value[i]+sumAlpha[key][i])//csps[0].cMI[csps[0].possibleVMtype.index(key)] for i in range(len(csps)) ]
            
    return activeNMI       

In [11]:
def auctionprocess(csps):
    auc = auctioneer(csps)
    auc.allbidsandasks()
    auc.sortBid()
#     auc.printbids()
    auc.winnerDetermination()
    auc.determineAlpha()
    auc.determineprice()
    return auc

In [ ]:
if __name__ =="__main__":
    print("    ROUND-0   \n")
    noofClouds = 10
    possibleVMtypes = ['ss','sm','sl','ls','lm','ll']
    possibleDS = [150,300]
    possiblenoofVM = [1,2]
    VMprice = [58.75,97,173.51,76.26,117.51,194] #values are in $/months(730 hours)
    VMpricepH = [ i/730 for i in VMprice]
    clouds = [i for i in range(noofClouds)]
    
    CSP = []

    for i in range(10):
        CSP.append(CSPs(i,np.random.randint(800,1000, size=6).tolist(),random.uniform(0.3,0.4),VMpricepH))
    

    timeslot = -1
    request = jobgenerator(data,timeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM)
    
    
    
#resource request per round for s type of job
    rsit = [[] for i in CSP]

    for req in request: 
        rsit[req.cloud].append(req)   
    
    for c in CSP:
        c.initqueue(rsit)
        c.getBids()
        
    auc= auctionprocess(CSP)
    
    muSIT = determineMu(CSP,auc)
#     print("\n# of jobs i->j:",muSIT)
    active=noOfactivatedServers(muSIT,CSP,auc)  
#     print("\n# of Active Servers:",active)
    for i in CSP:
        i.queueupdate(muSIT,rsit)
   


    timeperiod = 2000
    for t in range(timeperiod):
        print(f"\n      ROUND-{t+1}    \n ")
        newtimeslot = t%696
        request = jobgenerator(data,newtimeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM)

        rsit = [[] for i in CSP]

        for req in request: 
            rsit[req.cloud].append(req)   
    
        for c in CSP:
            c.getBids()

        auc= auctionprocess(CSP)
        
        muSIT = determineMu(CSP,auc)
#         print("\n# of jobs i->j:",muSIT)
        active=noOfactivatedServers(muSIT,CSP,auc)  
#         print("\n# of Active Servers:",active)
#updating the queue
        for i in CSP:
            i.dropjobs()
            i.vqueueupdate(muSIT,CSP)
            i.queueupdate(muSIT,rsit)
            i.provisioningcost(active)
            
    
#     temp = {} 
#     temp1={}
#     for i in CSP:
# #         print("total no. of job drop: ",i.pays)
        
# #         i.dropinend()
# #         print("quuelen :",i.quelen)
# #         print("total no. of job drop: ",i.pays)
#         i.totalpayment()
#         temp[f"CSP{i.name}"] = i.totalpay//timeperiod
#         temp1[f"CSP{i.name}"] = i.D
# #         print(f"total pay of {i.name} : ",i.totalpay//timeperiod)
# #         print(f"total pay of {i.name} : ",i.totalpay)
        

#     print(temp) 
    
    

    ROUND-0   

WinningBuyer: {'ss': {6: 0.0003863333333333333}, 'sm': {5: 0.00017066666666666668}, 'sl': {7: 8.1e-05}, 'ls': {1: 4.6e-05}, 'lm': {6: 0.0003713333333333333}, 'll': {4: 0.0005726666666666666}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-1    
 
WinningBuyer: {'ss': {6: 0.0007726666666666666}, 'sm': {5: 0.00034133333333333335}, 'sl': {7: 0.000162}, 'ls': {1: 9.2e-05}, 'lm': {6: 0.0007426666666666666}, 'll': {4: 0.0011453333333333333}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-2    
 
WinningBuyer: {'ss': {6: 0.0011343333333333333}, 'sm': {4: 0.0007096666666666667}, 'sl': {7: 0.0005103333333333333}, 'ls': {4: 0.0004206666666666667}, 'lm': {6: 0.0011576666666666667}, 'll': {4: 0.0014856666666666666}}
WinningSeller: {'ss': [[], 0], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-3    
 


      ROUND-26    
 
WinningBuyer: {'ss': {2: 0.012182}, 'sm': {9: 0.009796666666666667}, 'sl': {9: 0.009215666666666667}, 'ls': {1: 0.009208666666666667}, 'lm': {6: 0.010942333333333333}, 'll': {4: 0.012254333333333334}}
WinningSeller: {'ss': [[8, 6, 1, 9], 0.012181], 'sm': [[], 0], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-27    
 
WinningBuyer: {'ss': {4: 0.014903333333333333}, 'sm': {9: 0.010203333333333333}, 'sl': {7: 0.009586}, 'ls': {1: 0.009565333333333334}, 'lm': {6: 0.011377}, 'll': {5: 0.012963}}
WinningSeller: {'ss': [[8, 1, 6, 9, 2, 0, 5, 7], 0.014903333333333333], 'sm': [[], 0.01019], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-28    
 
WinningBuyer: {'ss': {7: 0.016071}, 'sm': {9: 0.010628333333333333}, 'sl': {7: 0.009950333333333334}, 'ls': {1: 0.009928666666666667}, 'lm': {6: 0.011800333333333333}, 'll': {4: 0.013532333333333334}}
WinningSeller: {'ss': [[4, 0, 8, 5, 3, 9, 2, 6], 0.016071], 'sm': [[8], 0.01059


      ROUND-49    
 
WinningBuyer: {'ss': {1: 0.01886583333333333}, 'sm': {2: 0.010730666666666666}, 'sl': {7: 0.009694333333333333}, 'ls': {1: 0.017519333333333335}, 'lm': {8: 0.020836}, 'll': {5: 0.029472333333333333}}
WinningSeller: {'ss': [[5, 9, 7, 0, 4, 6, 8, 3], 0.01886583333333333], 'sm': [[8, 1], 0.010707666666666667], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0]}

      ROUND-50    
 
WinningBuyer: {'ss': {3: 0.018985666666666668}, 'sm': {3: 0.010959666666666666}, 'sl': {7: 0.009892}, 'ls': {7: 0.017977666666666666}, 'lm': {8: 0.021396333333333333}, 'll': {2: 0.030588}}
WinningSeller: {'ss': [[1, 5, 9, 7, 0, 2, 4, 8], 0.018985666666666668], 'sm': [[8, 6, 1], 0.010959], 'sl': [[], 0], 'ls': [[], 0], 'lm': [[], 0], 'll': [[], 0.030548666666666665]}

      ROUND-51    
 
WinningBuyer: {'ss': {2: 0.019104333333333334}, 'sm': {4: 0.011179666666666666}, 'sl': {8: 0.010101333333333334}, 'ls': {7: 0.018347333333333334}, 'lm': {8: 0.021854666666666668}, 'll': {2: 0.0316


      ROUND-70    
 
WinningBuyer: {'ss': {7: 0.011620666666666666}, 'sm': {1: 0.0098715}, 'sl': {7: 0.012651833333333333}, 'ls': {9: 0.025288333333333333}, 'lm': {8: 0.029706666666666666}, 'll': {9: 0.032999}}
WinningSeller: {'ss': [[8, 6, 9], 0.011620666666666666], 'sm': [[], 0], 'sl': [[8, 6, 1, 9, 2, 0, 4, 5], 0.012651833333333333], 'ls': [[], 0], 'lm': [[], 0], 'll': [[8, 6], 0.0319256829974595]}

      ROUND-71    
 
WinningBuyer: {'ss': {7: 0.011857}, 'sm': {1: 0.010065333333333334}, 'sl': {4: 0.012837666666666667}, 'ls': {9: 0.025635666666666668}, 'lm': {8: 0.030250666666666665}, 'll': {3: 0.033702}}
WinningSeller: {'ss': [[8, 6, 9], 0.011857], 'sm': [[], 0], 'sl': [[8, 6, 1, 9, 2, 3, 7, 0], 0.012837666666666667], 'ls': [[], 0], 'lm': [[], 0], 'll': [[8, 6, 9], 0.03279566666666667]}

      ROUND-72    
 
WinningBuyer: {'ss': {1: 0.010236666666666666}, 'sm': {1: 0.010249333333333333}, 'sl': {5: 0.0130285}, 'ls': {9: 0.025974}, 'lm': {8: 0.030860333333333333}, 'll': {0: 0.033547

WinningBuyer: {'ss': {4: 0.010491333333333333}, 'sm': {7: 0.011222}, 'sl': {9: 0.010505333333333334}, 'ls': {4: 0.033205}, 'lm': {8: 0.027078333333333333}, 'll': {2: 0.03151166666666667}}
WinningSeller: {'ss': [[], 0.010491333333333333], 'sm': [[8, 1, 9], 0.010899333333333334], 'sl': [[], 0.010505333333333334], 'ls': [[8, 6, 1], 0.03220602232983989], 'lm': [[], 0], 'll': [[], 0.03047907347892455]}

      ROUND-92    
 
WinningBuyer: {'ss': {4: 0.010873}, 'sm': {0: 0.011157333333333333}, 'sl': {8: 0.010363166666666666}, 'ls': {3: 0.033551333333333336}, 'lm': {8: 0.027529666666666668}, 'll': {2: 0.031888}}
WinningSeller: {'ss': [[6, 1, 9], 0.010873], 'sm': [[8, 6, 1], 0.010735340776613296], 'sl': [[], 0], 'ls': [[8, 6, 1], 0.03220602232983989], 'lm': [[], 0], 'll': [[8], 0.03154265964000881]}

      ROUND-93    
 
WinningBuyer: {'ss': {8: 0.010786333333333334}, 'sm': {4: 0.011271166666666667}, 'sl': {1: 0.010138666666666667}, 'ls': {5: 0.028575333333333335}, 'lm': {8: 0.027945}, 'll': {0

...

WinningBuyer: {'ss': {5: 0.010982}, 'sm': {5: 0.010829166666666667}, 'sl': {3: 0.010751333333333333}, 'ls': {8: 0.03607866666666667}, 'lm': {7: 0.03567183333333333}, 'll': {6: 0.0329}}
WinningSeller: {'ss': [[8, 1, 9], 0.010982], 'sm': [[8, 1, 6], 0.010763333333333333], 'sl': [[8, 6, 1], 0.010735340776613296], 'ls': [[9, 1, 2], 0.035701166666666666], 'lm': [[9, 8, 1, 6], 0.03557772833401275], 'll': [[1, 9], 0.0329]}

      ROUND-1541    
 
WinningBuyer: {'ss': {6: 0.011087333333333333}, 'sm': {0: 0.010980666666666666}, 'sl': {1: 0.0109215}, 'ls': {3: 0.03586983333333333}, 'lm': {0: 0.035491166666666664}, 'll': {4: 0.032921}}
WinningSeller: {'ss': [[8, 1], 0.010735340776613296], 'sm': [[8, 1, 6], 0.010980666666666666], 'sl': [[8, 6], 0.010735340776613296], 'ls': [[9, 8, 6, 1], 0.03586983333333333], 'lm': [[9, 8, 1], 0.035491166666666664], 'll': [[6, 1, 9], 0.032921]}

      ROUND-1542    
 
WinningBuyer: {'ss': {3: 0.010960166666666667}, 'sm': {9: 0.011178333333333334}, 'sl': {2: 0.0108

WinningBuyer: {'ss': {2: 0.011236333333333333}, 'sm': {4: 0.011124833333333334}, 'sl': {8: 0.011027333333333333}, 'ls': {3: 0.035923333333333335}, 'lm': {8: 0.03571816666666667}, 'll': {4: 0.033285666666666665}}
WinningSeller: {'ss': [[8, 6, 1], 0.010735340776613296], 'sm': [[8, 1, 6], 0.010735340776613296], 'sl': [[6, 1], 0.011027333333333333], 'ls': [[1, 9, 2, 6], 0.035923333333333335], 'lm': [[1, 9, 2], 0.03571816666666667], 'll': [[6, 9, 8], 0.033285666666666665]}

      ROUND-1559    
 
WinningBuyer: {'ss': {7: 0.010981666666666667}, 'sm': {3: 0.011137}, 'sl': {9: 0.011062666666666667}, 'ls': {6: 0.03601083333333333}, 'lm': {6: 0.03574633333333333}, 'll': {4: 0.03347233333333333}}
WinningSeller: {'ss': [[6, 8, 1], 0.010857166666666666], 'sm': [[8, 1, 9], 0.010801833333333333], 'sl': [[8, 6], 0.0106418943324865], 'ls': [[1, 9, 2], 0.03601083333333333], 'lm': [[1, 8, 9], 0.03574633333333333], 'll': [[6, 9, 8], 0.03347233333333333]}

      ROUND-1560    
 
WinningBuyer: {'ss': {0: 0.

WinningBuyer: {'ss': {3: 0.0106515}, 'sm': {8: 0.010882333333333334}, 'sl': {0: 0.010612666666666666}, 'ls': {0: 0.03659816666666667}, 'lm': {8: 0.03586416666666667}, 'll': {5: 0.03370033333333333}}
WinningSeller: {'ss': [[8, 1], 0.0106515], 'sm': [[6, 9], 0.010882333333333334], 'sl': [[8], 0.010514219880002935], 'ls': [[8, 1, 2, 9], 0.036379], 'lm': [[6, 1, 2], 0.03558716666666667], 'll': [[8, 9, 6], 0.03352783333333333]}

      ROUND-1578    
 
WinningBuyer: {'ss': {3: 0.0108315}, 'sm': {8: 0.011103}, 'sl': {1: 0.010626166666666667}, 'ls': {5: 0.036463833333333334}, 'lm': {0: 0.0356685}, 'll': {7: 0.033745333333333336}}
WinningSeller: {'ss': [[8, 1, 9], 0.0108315], 'sm': [[6, 9], 0.011103], 'sl': [[8], 0.010514219880002935], 'ls': [[8, 1, 2, 9, 0], 0.036463833333333334], 'lm': [[6, 9, 8, 2], 0.0356685], 'll': [[8, 9, 6], 0.0337305]}

      ROUND-1579    
 
WinningBuyer: {'ss': {2: 0.01007}, 'sm': {5: 0.011060333333333333}, 'sl': {2: 0.010625166666666666}, 'ls': {6: 0.036502}, 'lm': {

WinningBuyer: {'ss': {6: 0.010868666666666667}, 'sm': {3: 0.010946333333333334}, 'sl': {5: 0.010677}, 'ls': {5: 0.03590683333333333}, 'lm': {8: 0.036111}, 'll': {8: 0.03508766666666667}}
WinningSeller: {'ss': [[8, 1], 0.010735340776613296], 'sm': [[6, 1, 9], 0.010946333333333334], 'sl': [[8, 6], 0.0106418943324865], 'ls': [[9, 8, 6, 2], 0.0358635], 'lm': [[2, 6, 9], 0.036111], 'll': [[6, 1], 0.034535333333333335]}

      ROUND-1597    
 
WinningBuyer: {'ss': {8: 0.010981}, 'sm': {9: 0.010595333333333333}, 'sl': {4: 0.010691333333333334}, 'ls': {0: 0.03596}, 'lm': {1: 0.036246}, 'll': {4: 0.03517783333333333}}
WinningSeller: {'ss': [[6, 1], 0.010735340776613296], 'sm': [[8], 0.010514219880002935], 'sl': [[8, 6], 0.0106418943324865], 'ls': [[9, 8, 2, 6], 0.03596], 'lm': [[8, 2, 6], 0.036246], 'll': [[8, 6, 1], 0.034777]}

      ROUND-1598    
 
WinningBuyer: {'ss': {5: 0.011086833333333334}, 'sm': {3: 0.010444}, 'sl': {2: 0.010545833333333334}, 'ls': {6: 0.036271}, 'lm': {3: 0.0359951666


      ROUND-1615    
 


In [ ]:
temp1={}
temp = {}
temp2 ={}
for i in CSP:
    i.totalpayment()
    temp[f"CSP{i.name}"] = i.totalpay//timeperiod
    temp1[f"CSP{i.name}"] = i.D
    temp2[f'CSP{i.name}'] = i.pays

In [ ]:
item = pd.DataFrame.from_dict(temp, orient='index')
item1 = pd.DataFrame.from_dict(temp1, orient='index')
item2 = pd.DataFrame.from_dict(temp2, orient='index')

writer = pd.ExcelWriter(f'./outputpay/paydata3_2000.xlsx', engine='xlsxwriter')
item.to_excel(writer,sheet_name="pay of all clouds")
item1.to_excel(writer,sheet_name="no of job drop")
item2.to_excel(writer,sheet_name="All Pays")
writer.save()

In [ ]:
for i in CSP:
    print(f"paysOff of ${i.name}:",i.pays)

In [ ]:
# for i in CSP:
#     print(i.D)



In [ ]:
# condition = data["timeslot"] < 101 
# req = data[condition]
# # print(df)
# condition =  req['sizecpu']=='s'
# req = req[condition].copy()
# condition =  req['sizeram']=='l'
# req = req[condition].copy()
# print(len(req))
